#### Dependecies

scipy = 0.17.0
gensim = 1.0.1
pandas = 0.19.2
numpy = 1.11.2

#### I used DATA_HADM_TOP10.csv from feature_extraction_nonseq.ipynb 

In [1]:
import pandas as pd
import pickle

df_hadm_top10 = pd.read_csv("/Volumes/EXTERNAL1/MIMICIII/DATA_HADM_TOP10.csv", escapechar='\\')
ICD9CODES = pickle.load(open("./data/ICD9CODES.p", "r"))

df_hadm_top10.head(5)

,id,4019,2724,25000,4280,41401,53081,51881,42731,5849,5990,text
0,117760,0,0,0,0,0,1,1,0,0,0,"""Admission Date: [**2118-12-14**] ..."
1,129030,1,1,0,0,0,1,0,0,0,0,Admission Date: [**2137-8-31**] ...
2,172040,0,0,0,0,1,0,0,0,1,0,Admission Date: [**2174-1-6**] D...
3,156170,0,0,1,1,0,0,0,1,1,0,Admission Date: [**2102-6-9**] D...
4,199180,0,0,1,1,1,0,0,0,0,0,Admission Date: [**2164-7-2**] Discharg...


In [2]:
len(df_hadm_top10)

52726

In [3]:
import random

def separate(seed, N):    
    idx=list(range(N))
    random.seed(seed)
    random.shuffle(idx)
    idx_train= idx[0:int(N*0.50)]
    idx_val= idx[int(N*0.50):int(N*0.75)]
    idx_test= idx[int(N*0.75):N]

    return idx_train, idx_val, idx_test


idx_train, idx_val, idx_test = separate(1234, df_hadm_top10.shape[0])
idx_join_train=idx_train + idx_val
len(idx_join_train)

39544

In [4]:
df_hadm_top10_d2v=df_hadm_top10.iloc[idx_join_train].copy()
df_hadm_top10_d2v.head(5)

,id,4019,2724,25000,4280,41401,53081,51881,42731,5849,5990,text
22653,112108,1,0,0,1,0,0,0,0,0,0,Admission Date: [**2113-3-2**] D...
14729,136683,1,0,0,0,0,0,0,0,0,0,Admission Date: [**2147-11-28**] Discharg...
17620,187756,0,0,1,1,0,1,0,1,0,0,Admission Date: [**2143-4-21**] ...
25598,108901,0,1,0,1,0,0,0,1,0,1,Admission Date: [**2169-1-14**] ...
48887,138443,0,0,0,0,0,0,0,0,0,0,Admission Date: [**2176-12-16**] ...


### Apply doc2vec

In [5]:
# Cleanning the data
# Light preprocesing done on purpose (so word2vec understand sentence structure)
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower())
    text = text.split()
    return text

token_review = list(df_hadm_top10_d2v['text'].apply(preprocessor))
len(token_review)

39544

In [13]:
#token_review[0]

In [6]:
import gensim
LabeledSentence = gensim.models.doc2vec.LabeledSentence
    
def labelizeReviews(reviews, label_type):
    labelized = []
    for i,v in enumerate(reviews):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized


In [7]:
sentence=labelizeReviews(token_review, "note")
len(sentence)

39544

In [61]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from gensim import utils
from time import time

# assumptions: window is 5 words left and right, eliminate words than dont occur in
# more than 10 docs, use 4 workers for a quadcore machine. Size is the size of vector
# negative=5 implies negative sampling and makes doc2vec faster to train
#model = Doc2Vec(sentence, size=100, window=5, workers=4, min_count=5)


import random

size = 600

#instantiate our DM and DBOW models
model_dm = Doc2Vec(min_count=10, window=5, size=size, sample=1e-3, negative=5, workers=4)
#model_dbow = Doc2Vec(min_count=10, window=5, size=size, sample=1e-3, negative=5, dm=0, workers=4)

#build vocab over all reviews
model_dm.build_vocab(sentence)
#model_dbow.build_vocab(sentence)

#We pass through the data set multiple times, shuffling the training reviews each time to improve accuracy.
Idx=list(range(len(sentence)))

t0 = time()
for epoch in range(5):
     random.shuffle(Idx)
     perm_sentences = [sentence[i] for i in Idx]
     model_dm.train(perm_sentences)
     print(epoch)
    
elapsed=time() - t0
print("Time taken for Doc2vec training: ", elapsed, "seconds.")

0
1
2
3
4
('Time taken for Doc2vec training: ', 1227.992525100708, 'seconds.')


In [220]:
#Infer a vector for new text => input a list of tokens
#model_dm.infer_vector(['the', 'patient', 'is', 'dead', '.'])


In [237]:
import gensim
# saves the doc2vec model to be used later.
#model_dm.save('./data/model_doc2vec_v2_600dim')

# open a saved doc2vec model 
model_dm=gensim.models.Doc2Vec.load('./data/model_doc2vec_v2_600dim')

#model_dm.wv.save_word2vec_format('./data/model_doc2vec_v2_600dim.txt', binary=False)


In [49]:
def preprocessor_2(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower())
    #text = text.split()
    return text

test_set_clean=df_hadm_top10.iloc[idx_test]['text'].apply(preprocessor_2)

In [238]:
model_dm_600dim=test_set_clean.apply(model_dm.infer_vector)

In [239]:
import numpy as np

train_d2v=np.asarray(model_dm.docvecs)
df_train_d2v = pd.DataFrame(data=train_d2v, index=idx_join_train)
test_d2v = [tuple(x) for x in model_dm_600dim]
df_test_d2v = pd.DataFrame(data=test_d2v, index=idx_test)
df_d2v = pd.concat([df_train_d2v,df_test_d2v])


In [240]:
#len(model_dm.docvecs)
#test_set_clean.iloc[0]
len(df_d2v)

52726

In [242]:
#save to drive            
df_d2v.to_csv("model_doc2vec_v2_600dim_final.csv")

In [241]:
# Find the 10 most similar words
model_dm.most_similar('melanoma')

[('sarcoma', 0.3752405643463135),
 ('adenocarcinoma', 0.36460477113723755),
 ('cancer', 0.34577974677085876),
 ('carcinoma', 0.33788567781448364),
 ('melenoma', 0.3214113116264343),
 ('mole', 0.3124437928199768),
 ('leiomyosarcoma', 0.3108573257923126),
 ('tumor', 0.3053992688655853),
 ('nevus', 0.30361607670783997),
 ('lipoma', 0.29490727186203003)]

In [243]:
# Find 10 most similar docs
model_dm.docvecs.most_similar('note_0')

[('note_18392', 0.4303767681121826),
 ('note_22083', 0.4224788248538971),
 ('note_8182', 0.4058257043361664),
 ('note_28179', 0.394045352935791),
 ('note_7600', 0.3888844847679138),
 ('note_2749', 0.3831929564476013),
 ('note_34920', 0.3824225664138794),
 ('note_18099', 0.37785446643829346),
 ('note_20967', 0.3716892898082733),
 ('note_790', 0.37038496136665344)]